In [ ]:
import mysql.connector
from mysql.connector import Error
import tkinter as tk
from tkinter import ttk
from tkinter import scrolledtext
import nltk as n
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from textblob import TextBlob

class Ufs: 
    def __init__(self):
        self.mydb = mysql.connector.connect(
           host="localhost",
           user="root",
           password="Hari@25082002",
           database="library_database")
        self.cursor = self.mydb.cursor()
        self.fornt_page = tk.Tk()
        self.fornt_page.title("  UFA  ")
        self.fornt_page.configure(bg="white")
        self.style = ttk.Style()
        self.style.configure("Custom.TFrame", background="white",highlightbackground="blue", highlightthickness=2) 
        self.frame = ttk.Frame(self.fornt_page, borderwidth=15, relief="raised",style="Custom.TFrame")
        self.frame.pack()
        title = tk.Label(self.frame, text=" UFA ", font=("Algerian", 50, 'bold'), bg="white")
        title.grid(row=0, columnspan=2, padx=20, pady=20)
        self.output = scrolledtext.ScrolledText(self.frame, wrap=tk.WORD, width=60, height=20,font=("Arial", 12),relief="raised", borderwidth=3)
        self.output.grid(row=1, column=0, columnspan=2, sticky="nsew")
        self.output.config(state="disabled")
        self.entry = tk.Entry(self.frame,width=50,relief="raised", borderwidth=3,font=("Arial", 14))
        self.entry.insert(0, "Enter your roll number or Hall ticket Number")
        self.entry.bind("<FocusIn>", self.on_entry_focus_in)
        self.entry.bind("<FocusOut>", self.on_entry_focus_out)
        self.entry.grid(row=2, column=0, padx=(10, 5), pady=10)
        submit = tk.Button(self.frame, text="go", font=("Algerian", 12),command=self.submit_clicked)
        submit.grid(row=2, column=1, padx=(5, 10), pady=10)
        self.entry.bind("<Return>", self.submit_clicked)
        self.name=False
        self.id=None
        self.initial_id = None
        self.fornt_page.mainloop()      
    def on_entry_focus_in(self, event):
        placeholder_text = "Enter your roll number or Hall ticket Number"
        if self.entry.get() == placeholder_text:
            self.entry.delete(0, tk.END)          
    def on_entry_focus_out(self, event):
        if not self.entry.get():
            self.entry.insert(0, "Enter your roll number or Hall ticket Number")     
    def submit_clicked(self, event=None):
        self.std_id = self.entry.get()
        if self.name == False:
            self.initial_id = self.std_id
            self.id=self.std_id
            self.output.config(state="normal")
            self.output.insert(tk.END, f"You: {self.std_id}\n")
            self.output.config(state="disabled")
            try:
                query = "select std_name from student_database where std_id=%s"
                self.cursor.execute(query, (self.std_id,))
                name = self.cursor.fetchone()[0]
                welcome_message = f"UAF : Hi {name} Welcome To UAF How Can I  Help You Today \n"
                for i, char in enumerate(welcome_message):
                     self.output.after(i*50, self.insert_char, char)
                self.name=True
            except mysql as error:
                print(error)
            finally:
                self.cursor.close()
                self.mydb.close()
        if self.name:
            self.start_chat()
        if self.std_id=="clear":
            self.output.config(state="normal")  # Set the state to normal to enable editing
            self.output.delete(1.0, tk.END)
            self.output.config(state="disabled")
    def start_chat(self):
        if self.id==self.std_id:
           self.entry.delete(0, tk.END)
           self.output.config(state="disabled")
           return
        self.entry.delete(0, tk.END)
        student_query = self.std_id
        self.output.config(state="normal")
        self.output.insert(tk.END, f"UAF: {student_query}\n")
        self.output.config(state="disabled")
        self.entry.config(state="disabled")
        self.preprocess_query(student_query)
    def preprocess_query(self,query):
        print("input qurey =",query)
        token = n.word_tokenize(query)
        print("token",token)
        lemma=WordNetLemmatizer()
        lemmatized_words = [lemma.lemmatize(word) for word in token] 
        print("lema",lemmatized_words)
        stop_words = set(stopwords.words('english')) 
        filtered_words = [word for word in lemmatized_words if word.lower() not in stop_words]
        print("stop words", filtered_words)
        words=str(filtered_words)
        blob=TextBlob(words)
        final=blob.correct()
        print("final querry = ",final)
        self.recognize_intent(final)  
    def recognize_intent(self,final):
        book_keywords= ["many","have","taken","can","return","not return","available","book"]
        info=["subject","year","1","2","3","4"]
        library_info = ["history","fine"]
        for keyword in book_keywords:
         if keyword in final:
            intent="Book Availability"
            self.generate_response(intent, final)
            print("Book Availability")
            break
        for keyword in info:
          if keyword in final:
            intent="info"
            self.generate_response(intent, final)
            print("info")
            break       
        for keyword in library_info:
          if keyword in final:
            intent="library_info"
            self.generate_response(intent, final)
            print("library_info")
            break
        intent="some thing"
        self.generate_response(intent, final)
    def generate_response(self,intent, final):
     if intent=="Book Availability":
      many="many" in final
      have='have' in final
      take='take'in final
      can="can" in final
      retur="return" in final
      not_return="not return" in final
      available="available" in final
      book="book"in final
      try:
        mydb, cursor = self.establish_connection()
        print(self.initial_id)
        if many and have and take and can and retur and not_return and available:
            final_query= "not there "
            print(final_query)
            return
        if (many or have) and (take or can):
               final_query="select num_books_borrowed from student_database where std_id=%s"
               cursor.execute(final_query, (self.initial_id,))
               name=cursor.fetchone()[0]
               if name==0:
                  response="Till now u have not taken any books so u can take Upto 3 books now"
               else:
                   answer=3-name
                   response=f"U have taken {name} books already so u can take only {answer} books now "
               self.data_base_query(response)
               return
        if (have or many) or (retur or not_return):
              final_query="select return_on from issue_book where student_id=%s"
              cursor.execute(final_query, (self.initial_id,))
              name=cursor.fetchone()[0]
              if name==None:
                  final_query="select return_date,book_id from issue_book where student_id=%s"
                  cursor.execute(final_query, (self.initial_id,))
                  name=cursor.fetchone()
                  return_date, book_id = name
                  response=f"A book is issued for u with a book_id={book_id},it should be returned on {return_date} so return before  the return date to avoide fine"
              else:
                  final_query="select return_on,book_id,fine from issue_book where student_id=%s"
                  cursor.execute(final_query, (self.initial_id,))
                  name=cursor.fetchone()
                  return_date, book_id , fine = name
                  if fine==0:
                      fine_statement="with out any fine"
                  else:
                      fine_statement=f" u have paid a fine of {fine} ruppes"
                  response=f"You u have returned a book  book_id={book_id} on {return_date} {fine_statement}"  
              self.data_base_query(response)
              return 
        if available or book:
              final_query="select status from books_database where book_name=%s or book_author=%s" 
              self.cursor.execute(final_query, (self.initial_id,))
              name=self.cursor.fetchone()[0]
              response=name
              print(response)
              self.data_base_query(response)
              return
      except mysql.connector.Error as error:
         self.handle_mysql_error(error)
         return
      finally:
         if 'cursor' in locals() and cursor:
                cursor.close()
         if 'mydb' in locals() and mydb.is_connected():
                mydb.close()
     if intent=="info":
        subject="subject" in final
        year= "year" in final
        one="1" in final
        two="2" in final
        three="3" in final
        four="4" in final
        try:
            mydb, cursor = self.establish_connection()
            print(self.initial_id)
            if subject and year and one and two and three and four:
              final_query="unabe to process"
              print(final_query)
              self.data_base_query(final_query)
              return 
            if subject:
               final_query="select book,refernce from  books_uaf where subject=%s"
               cursor.execute(final_query, (self.initial_id,))
               name=cursor.fetchone()[0]
               response=name
               self.data_base_query(response)
               return 
            if year:
               final_query="select subject,book,refernce from  books_uaf where year=%s"
               cursor.execute(final_query, (self.initial_id,))
               name=cursor.fetchone()[0]
               response=name
               self.data_base_query(response)   
               return final_query
        except mysql.connector.Error as error:
               self.handle_mysql_error(error)
               return
        finally:
            if 'cursor' in locals() and cursor:
                cursor.close()
            if 'mydb' in locals() and mydb.is_connected():
                mydb.close()
     if intent=="library_info":
        history="history" in final
        fine="fine" in final 
        try:
            mydb, cursor = self.establish_connection()
            print(self.initial_id)
            if history:
               final_query="""
                  SELECT i.issue_id, i.book_id ,b.book_name, b.book_author, b.book_published_year, s.std_name, s.branch,  s.year, s.phone_number, i.issue_date, i.return_date,i.return_on, i.fine
                  FROM issue_book i
                  JOIN books_database b ON i.book_id = b.book_id
                  JOIN student_database s ON i.student_id = s.std_id
                  WHERE i.student_id = %s """
               cursor.execute(final_query, (self.initial_id,))
               name=cursor.fetchone()[0]
               response=name
               self.data_base_query(response)
               return 
            if fine:
              final_query="select fine from  issue_book where student_id=%s"
              cursor.execute(final_query, (self.initial_id,))
              name=cursor.fetchone()[0]
              response=name
              self.data_base_query(response)
              return 
        except mysql.connector.Error as error:
           self.handle_mysql_error(error)
           return
        finally:
            if 'cursor' in locals() and cursor:
                cursor.close()
            if 'mydb' in locals() and mydb.is_connected():
                mydb.close()      
    def data_base_query(self,response):
        print("its called ")
        self.output.config(state="normal")
        for i, char in enumerate(response):
          self.output.after(i*50, self.insert_char, char)
        self.output.config(state="disabled")
    def insert_char(self, char):
        self.output.config(state="normal")
        self.output.insert(tk.END, char)
        self.output.see(tk.END) 
    def establish_connection(self):
       try:
          mydb = mysql.connector.connect(
             host="localhost",
             user="root",
             password="Hari@25082002",
             database="library_database")
          cursor = mydb.cursor()
          return mydb, cursor
       except mysql.connector.Error as error:
          print("Error while connecting to MySQL", error)
          raise  
    def handle_mysql_error(self,error):
     if isinstance(error, mysql.connector.IntegrityError):
        print("IntegrityError occurred:", error)
     elif isinstance(error, mysql.connector.ProgrammingError):
        print("ProgrammingError occurred:", error)
     elif isinstance(error, mysql.connector.DataError):
        print("DataError occurred:", error)
     elif isinstance(error, mysql.connector.DatabaseError):
        print("DatabaseError occurred:", error)
     elif isinstance(error, mysql.connector.InterfaceError):
        print("InterfaceError occurred:", error)
     elif isinstance(error, mysql.connector.InternalError):
        print("InternalError occurred:", error)
     elif isinstance(error, mysql.connector.OperationalError):
        print("OperationalError occurred:", error)
     else:
        print("An unexpected error occurred:", error)

my = Ufs()
